Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [123]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Folder dir for saving and loading files

In [124]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
model_name = 'mtm_spring'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [125]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

MTM
['Coulomb', 'viscous', 'offset']


In [126]:
robot_def.bary_params

## Create kinematics chain

In [127]:
geom = Geometry(robot_def)

In [128]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [129]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/10
Calculating the link kinetic energy of 2/10
Calculating the link kinetic energy of 3/10
Calculating the link kinetic energy of 4/10
Calculating the link kinetic energy of 5/10
Calculating the link kinetic energy of 6/10
Calculating the link kinetic energy of 7/10
Calculating the link kinetic energy of 8/10
Calculating the link kinetic energy of 9/10
Calculating joint torques...
tau of q1
tau of q2
tau of q8
tau of q4
tau of q5
tau of q6
tau of q7
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q8, q4, q5, q6, q7, dq1, dq2, dq8, dq4, dq5, dq6, dq7, ddq1, ddq2, ddq8, ddq4, ddq5, ddq6, ddq7))
Calculating M, C and G...
Calculating base parameter...
[ 79  80  82  66  67  69  78  83  84  85  53  71  72  56  54  81  59  58
  40  46  70  45  32  47 110  41 111  43  57  14  52  27  55  15  37  44
  30  28  17  16  42  97  98  29  89  18  50  76  63  11  24 102 119  91
  9

In [130]:
robot_def.bary_params

In [131]:
sympy.Matrix(dyn.base_param)

⎡                                                                  1.0⋅L7xy   
⎢                                                                             
⎢                                                                  1.0⋅L7xz   
⎢                                                                             
⎢                                                                  1.0⋅L7yz   
⎢                                                                             
⎢                                                                  1.0⋅L6xy   
⎢                                                                             
⎢                                                                  1.0⋅L6xz   
⎢                                                                             
⎢                                                                  1.0⋅L6yz   
⎢                                                                             
⎢                                                   

In [132]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

Saved 70 parameters


---
# 3 - Excitation trajectory optimization

In [133]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
model_name = 'mtm_spring'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [134]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [135]:
trajectory_name = 'one'
# trajectory_name = 'two_normalized'

In [137]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm' or model_name == 'mtm_spring':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 8
cartesian constraint number: 0
constraint number: 32
frames_constrained: []
cond: 130.696730038
cond: 130.696730038
cond: 130.696729731
cond: 130.696730389
cond: 130.696731848
cond: 130.696733898
cond: 130.696734109
cond: 130.696735978
cond: 130.696736754
cond: 130.696734977
cond: 130.696734256
cond: 130.696732932
cond: 130.696732697
cond: 130.696731241
cond: 130.696727554
cond: 130.696725146
cond: 130.696725923
cond: 130.696726754
cond: 130.696728833
cond: 130.69672865
cond: 130.69673045
cond: 130.696734783
cond: 130.696733256
cond: 130.696731342
cond: 130.696730045
cond: 130.696729358
cond: 130.696728975
cond: 130.696704843
cond: 130.696704695
cond: 130.69672953
cond: 130.696724745
cond: 130.69670845
cond: 130.696736712
cond: 130.696710157
cond: 130.696725409
cond: 130.696759126
cond: 130.696741988
cond: 130.696745045
cond: 130.696743802
cond: 130.696735937
cond: 130.6967704
cond: 130.696742092
cond: 130.696742805
cond: 130.696743895
cond: 130.696745326
con

cond: 142.845211753
cond: 142.845106137
cond: 142.845144846
cond: 142.845157589
cond: 142.845135658
cond: 142.845170035
cond: 142.845167657
cond: 142.845174913
cond: 142.845168946
cond: 142.845172158
cond: 142.845172952
cond: 142.8451719
cond: 142.845178298
cond: 142.84516546
cond: 142.84517811
cond: 142.845172566
cond: 142.845174059
cond: 142.845174546
cond: 142.845174939
cond: 142.845169455
cond: 142.845172714
cond: 142.845169962
cond: 142.845170226
cond: 142.84517332
cond: 142.845172876
cond: 142.845174504
cond: 142.845171446
cond: 142.845173362
cond: 142.845172053
cond: 142.845172266
cond: 142.845171785
cond: 142.845171768
cond: 142.845169163
cond: 142.845171606
cond: 142.84516934
cond: 142.845171553
cond: 142.845172762
cond: 142.845173015
cond: 142.845175868
cond: 142.845173122
cond: 142.845173733
cond: 142.845172465
cond: 142.845172358
cond: 142.845170943
cond: 142.845172159
cond: 142.845171962
cond: 142.845171902
cond: 142.84517195
cond: 142.84517182
cond: 142.845171893
cond: 14

cond: 295.763750099
cond: 295.763758327
cond: 295.763762126
cond: 295.763760047
cond: 295.763762109
cond: 295.763762431
cond: 295.763762667
cond: 295.763761947
cond: 295.763761916
cond: 295.763761898
cond: 295.763761892
cond: 295.763760385
cond: 295.763761067
cond: 295.76376155
cond: 295.763761974
cond: 285.391778316
cond: 271.343992694
cond: 271.343992694
cond: 271.343973971
cond: 271.343982642
cond: 271.343988732
cond: 271.344023916
cond: 271.343946438
cond: 271.34399049
cond: 271.344002719
cond: 271.34401065
cond: 271.343950651
cond: 271.343997063
cond: 271.344017119
cond: 271.343931583
cond: 271.343982697
cond: 271.343993317
cond: 271.343855053
cond: 271.343862045
cond: 271.344057395
cond: 271.343801617
cond: 271.344034273
cond: 271.343983452
cond: 271.343970906
cond: 271.34394753
cond: 271.344004691
cond: 271.344118417
cond: 271.344010547
cond: 271.344043211
cond: 271.343767608
cond: 271.344177529
cond: 271.344149241
cond: 271.343910358
cond: 271.344156932
cond: 271.343907605
cond

cond: 229.807530417
cond: 229.807572906
cond: 229.807583861
cond: 229.807536926
cond: 229.80755569
cond: 229.807190334
cond: 229.807738157
cond: 229.80761938
cond: 229.807447065
cond: 229.807708406
cond: 229.807456594
cond: 229.807563508
cond: 229.807666413
cond: 229.807504923
cond: 229.807567816
cond: 229.807479664
cond: 229.807504711
cond: 229.807534918
cond: 229.807522442
cond: 229.807520222
cond: 229.807532435
cond: 229.807530514
cond: 229.8075287
cond: 229.807522198
cond: 229.807545402
cond: 229.807534461
cond: 229.807530824
cond: 229.807531212
cond: 229.807528394
cond: 229.807530098
cond: 229.807527159
cond: 229.807529554
cond: 229.807528077
cond: 229.807526167
cond: 229.807530799
cond: 229.807528597
cond: 229.80753421
cond: 229.807548248
cond: 229.80753737
cond: 229.807528799
cond: 229.807536256
cond: 229.807528789
cond: 229.807531786
cond: 229.807531399
cond: 229.807533591
cond: 229.807532139
cond: 229.807531571
cond: 229.807531881
cond: 229.807531228
cond: 229.807531475
cond: 

cond: 211.830948166
cond: 211.830945662
cond: 211.830945646
cond: 211.830947217
cond: 211.830944535
cond: 211.830953088
cond: 211.830945967
cond: 211.830942962
cond: 211.830948221
cond: 211.830947059
cond: 211.830941749
cond: 211.83094781
cond: 211.830945896
cond: 211.830947018
cond: 211.830947516
cond: 211.830945957
cond: 211.830946489
cond: 211.830945949
cond: 211.830947243
cond: 211.830945952
cond: 211.830946407
cond: 211.830946416
cond: 211.830946653
cond: 211.830947166
cond: 211.830945679
cond: 211.830946425
cond: 211.830946217
cond: 211.830946294
cond: 211.83094616
cond: 235.767412592
cond: 216.799320754
cond: 209.670554724
cond: 209.670554724
cond: 209.670542874
cond: 209.670541306
cond: 209.670555689
cond: 209.670568168
cond: 209.670533721
cond: 209.670543949
cond: 209.670570872
cond: 209.670570967
cond: 209.670530784
cond: 209.670569912
cond: 209.670570214
cond: 209.67050984
cond: 209.67054737
cond: 209.670677537
cond: 209.670429456
cond: 209.670473059
cond: 209.670638285
cond

...

cond: 201.315626866
cond: 201.315627014
cond: 201.315626595
cond: 201.315626669
cond: 201.315626728
cond: 201.315626513
cond: 201.315627183
cond: 201.315626635
cond: 201.315626553
cond: 201.315626801
cond: 211.269129459
cond: 201.350358346
cond: 200.920224738
cond: 200.920224738
cond: 200.920218479
cond: 200.920214696
cond: 200.920223641
cond: 200.920238035
cond: 200.920205462
cond: 200.920215789
cond: 200.920243482
cond: 200.920246707
cond: 200.920196543
cond: 200.920239037
cond: 200.920237941
cond: 200.920179423
cond: 200.920208465
cond: 200.92033769
cond: 200.920077903
cond: 200.920150555
cond: 200.920308929
cond: 200.920074336
cond: 200.920251037
cond: 200.920214174
cond: 200.920180752
cond: 200.920224041
cond: 200.920256858
cond: 200.920280442
cond: 200.92022933
cond: 200.920243102
cond: 200.919956286
cond: 200.920438522
cond: 200.920303962
cond: 200.920142583
cond: 200.920350355
cond: 200.920186685
cond: 200.920218429
cond: 200.920346779
cond: 200.920198019
cond: 200.920224199
co

cond: 199.95385122
cond: 199.953869042
cond: 199.953553605
cond: 199.954049814
cond: 199.953912548
cond: 199.953776293
cond: 199.953961819
cond: 199.953809809
cond: 199.953843644
cond: 199.953972098
cond: 199.953821796
cond: 199.95384769
cond: 199.953785104
cond: 199.953841493
cond: 199.953846208
cond: 199.953848815
cond: 199.95384864
cond: 199.953847271
cond: 199.953846607
cond: 199.953846375
cond: 199.953847047
cond: 199.953846634
cond: 199.95384646
cond: 199.953845806
cond: 199.953847075
cond: 199.953846772
cond: 199.953848536
cond: 199.953847007
cond: 199.953848204
cond: 199.953846861
cond: 199.953847844
cond: 199.953847611
cond: 199.953847057
cond: 199.953847477
cond: 199.953848658
cond: 199.953848302
cond: 199.953846713
cond: 199.953847598
cond: 199.953847928
cond: 199.953847238
cond: 199.953847396
cond: 199.95384838
cond: 199.953847007
cond: 199.953848209
cond: 199.95384776
cond: 199.953847687
cond: 199.953847416
cond: 199.953845342
cond: 199.953848314
cond: 199.953848079
cond: 

cond: 199.326220494
cond: 199.326219717
cond: 199.326222802
cond: 199.326221249
cond: 199.326219967
cond: 199.326220948
cond: 199.326220897
cond: 199.326220903
cond: 199.326220126
cond: 199.326220023
cond: 199.326221903
cond: 199.326221797
cond: 199.326221023
cond: 199.326220903
cond: 199.326221067
cond: 199.326220938
cond: 199.326220845
cond: 199.326220842
cond: 199.326220932
cond: 199.32622095
cond: 199.326220973
cond: 199.326220864
cond: 199.326220856
cond: 199.326220945
cond: 199.326220925
cond: 199.326220907
cond: 203.841841082
cond: 199.191044649
cond: 199.191044649
cond: 199.191038118
cond: 199.191035112
cond: 199.191041605
cond: 199.191058705
cond: 199.191021836
cond: 199.191034819
cond: 199.191064329
cond: 199.191066218
cond: 199.191020213
cond: 199.191060133
cond: 199.191051894
cond: 199.19100112
cond: 199.191030785
cond: 199.191159266
cond: 199.190895672
cond: 199.190972582
cond: 199.191129609
cond: 199.190894631
cond: 199.191070829
cond: 199.191035669
cond: 199.190999945
co

cond: 198.889265628
cond: 198.889296827
cond: 198.88942567
cond: 198.889157254
cond: 198.889236651
cond: 198.889394179
cond: 198.889159253
cond: 198.889335726
cond: 198.889298388
cond: 198.88926309
cond: 198.889310106
cond: 198.889342654
cond: 198.889366404
cond: 198.889313926
cond: 198.889327507
cond: 198.889032579
cond: 198.889518937
cond: 198.889381939
cond: 198.889229019
cond: 198.889430879
cond: 198.8892707
cond: 198.889304353
cond: 198.889432795
cond: 198.889281988
cond: 198.889310346
cond: 198.88925659
cond: 198.889300673
cond: 198.889309003
cond: 198.889310666
cond: 198.889310726
cond: 198.889309413
cond: 198.889308279
cond: 198.88930858
cond: 198.889308891
cond: 198.889310389
cond: 198.88930794
cond: 198.889308674
cond: 198.889308237
cond: 198.889309833
cond: 198.889310445
cond: 198.889309082
cond: 198.889312879
cond: 198.889309633
cond: 198.889309408
cond: 198.889309074
cond: 198.889309327
cond: 198.889309772
cond: 198.889310151
cond: 198.889310471
cond: 198.889309392
cond: 1

cond: 198.567224936
cond: 198.567224106
cond: 198.567223975
cond: 198.567224094
cond: 198.567224555
cond: 198.567224912
cond: 198.567221856
cond: 198.567224323
cond: 198.56722465
cond: 198.567224
cond: 198.567224368
cond: 198.567224543
cond: 198.567224004
cond: 198.567225079
cond: 198.567224088
cond: 198.567224566
cond: 198.567224401
cond: 198.567224318
cond: 198.567224407
cond: 198.567224016
cond: 198.56722471
cond: 198.567224157
cond: 198.567224322
cond: 198.567224215
cond: 198.567224283
cond: 198.567224314
cond: 198.567224169
cond: 198.567224242
cond: 198.567224287
cond: 198.567224366
cond: 198.567224321
cond: 198.56722427
cond: 198.567224219
cond: 198.56722429
cond: 198.567224379
cond: 198.567224322
cond: 198.567224229
cond: 198.567224279
cond: 201.223095042
cond: 198.498147846
cond: 198.498147846
cond: 198.498141919
cond: 198.498139148
cond: 198.498146448
cond: 198.498160746
cond: 198.498127357
cond: 198.498138614
cond: 198.498165912
cond: 198.498170175
cond: 198.498123422
cond: 1

In [138]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [139]:
traj_optimizer.calc_frame_traj()

In [140]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [141]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [142]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.2414,0.0067,-0.0019,-0.0071,-0.0007,0.008,0.7491,-0.0006,-0.014,0.0018,-0.0003,-0.0017,2.7823
q2,0.3865,0.0612,-0.0987,0.1611,-0.1103,1.3787,0.2411,-0.026,-0.0576,0.1485,-0.5554,-0.3407,-0.0747
q8,0.2805,0.0946,-0.2565,-0.2301,0.133,0.3449,0.3359,0.0092,-0.2255,0.0786,-0.2744,0.2038,0.0568
q4,0.9192,0.0444,0.7513,1.3374,-0.471,0.1087,-0.6092,0.2326,0.7052,1.8321,-0.339,0.235,-2.7182
q5,1.2311,-0.75,0.8664,-0.1135,1.3938,0.1365,-2.0372,-0.1502,0.2958,-0.1806,0.07,0.8192,-0.2739
q6,-0.0714,0.0277,-0.1577,-0.4567,0.0703,0.1402,-0.2456,0.1268,-0.0566,0.1979,-0.3805,0.8409,0.3349
q7,-2.9053,-0.6684,-0.8079,-3.0967,-3.7475,2.6571,7.4342,-0.8593,3.7604,6.0992,4.2045,-1.9045,-4.7674


---
# Data processing

In [99]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

In [100]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
model_name = 'mtm'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

# trajectory_name = 'one'
# trajectory_name = 'two'
trajectory_name = 'two_normalized'

In [101]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
dof, fourier_order, base_freq, traj_optimizer_result, reg_norm_mat = load_data(trajectory_folder, trajectory_name)


trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_data(model_folder,model_name)

(<type 'numpy.ndarray'>, (14989, 21))


## Calculate filter cut-off frequency

In [102]:
fc_mult = 4.0
# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 2.4


## Get velocity and acceleration data and filter raw data

In [103]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (14989, 7)
0
1
2
3
4
5
6
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [104]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [105]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((102123, 69), (102123,))
9969.298840690999


In [106]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [107]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,-2.3079974732776946e-05
1.0*L7xz,1.7201011186607196e-05
1.0*L7yz,-6.64655577425192e-05
1.0*L6xy,-0.0003114409863463382
1.0*L6yz,0.0001092921409935636
1.0*L6xz,-0.00033444365976893244
1.0*L6xx - 1.0*L6yy + 1.0*L7yy,0.0001762183500464351
1.0*L5xy,-0.000855112348396251
1.0*L7zz,8.538421064531801e-05
1.0*l7y,0.00047861865326217275


## Compare measured torque and predicted torque

### Generate predicted torque

In [108]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [109]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.0003805930424101266
standard deviation of xb using OLS:
[1.41605663e-06 8.51387970e-07 8.35816171e-07 7.45747576e-06
 8.50819791e-06 6.64052589e-06 1.79719645e-05 9.66010630e-06
 2.61594436e-06 2.60458955e-06 2.62574485e-06 2.27643170e-05
 8.88314472e-06 3.16206475e-05 1.78946236e-05 8.73881756e-06
 3.17477438e-05 1.12334680e-05 4.03153569e-05 2.38698976e-05
 1.26338207e-05 1.94425776e-05 2.84516503e-05 1.29227709e-05
 6.86739210e-04 2.14698290e-03 3.93434585e-04 1.36437220e-04
 1.84512343e-05 1.29315208e-04 2.96036992e-04 1.78464440e-05
 1.87251740e-05 2.24818938e-04 3.02342977e-04 7.66119681e-05
 3.77398720e-05 1.02608586e-04 4.51538102e-05 4.47910468e-04
 2.20932660e-04 7.35074010e-04 7.08852195e-04 2.33384881e-04
 4.53197112e-04 9.98471553e-05 4.76583056e-04 7.12900416e-05
 3.10286541e-05 9.33834204e-05 4.48393508e-04 4.04588633e-04
 3.48682867e-03 4.49657449e-04 1.75513225e-04 1.62208559e-04
 1.69628447e-04 1.78638792e-04 1.61876175e-04 2.

### Plot measured torque and predicted torque

In [111]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])

## Weighted Least Square (WLS)

In [112]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.02701119, 0.04529386, 0.03864301, 0.01879335, 0.01449041,
       0.01099942, 0.00584319])

In [113]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.67470093, 5.52178786, 3.26941497, 0.46762431, 0.3529792 ,
       0.10158411, 0.00586076])

In [114]:
W_b.shape, tau_s.shape

In [115]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [116]:
weights.shape

In [117]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [118]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [119]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [120]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [121]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

In [122]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.07462344, 0.34174096, 0.19525647, 0.34850154, 0.17395656,
       0.19796057, 0.25425048])

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/matplotlib/tight_layout.py:209: UserWarning: tight_layout cannot make axes height small enough to accommodate all axes decorations
  warnings.warn('tight_layout cannot make axes height small enough '


## Convex optimization

### Generate regressor matrix for barycentric parameters

In [149]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [150]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [151]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [173]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [174]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 120)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4268296, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
Variables n = 121, constraints m = 102447
Cones:	linear vars: 133
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.62e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.50

  9400| 3.10e-01  1.06e-02  5.74e-04  1.25e+02  1.26e+02  5.87e-14  1.63e+03 
  9500| 3.10e-01  1.06e-02  5.65e-04  1.27e+02  1.27e+02  5.98e-14  1.65e+03 
  9600| 3.09e-01  1.00e-02  5.54e-04  1.28e+02  1.28e+02  6.08e-14  1.67e+03 
  9700| 3.09e-01  1.02e-02  5.43e-04  1.29e+02  1.29e+02  6.19e-14  1.69e+03 
  9800| 3.08e-01  1.05e-02  5.35e-04  1.30e+02  1.30e+02  6.30e-14  1.70e+03 
  9900| 3.08e-01  9.87e-03  5.24e-04  1.31e+02  1.32e+02  6.40e-14  1.72e+03 
 10000| 3.08e-01  9.73e-03  5.15e-04  1.33e+02  1.33e+02  6.51e-14  1.74e+03 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate
Timing: Solve time: 1.74e+03s
	Lin-sys: avg # CG iterations: 35.15, avg solve time: 1.72e-01s
	Cones: avg projection time: 2.59e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 6.4829e-09, dist(y, K*) = 2.7388e-09, s'y/|s||y| = 2.0676e-11
|Ax + s -

## Compare measured torque and predicted torque

### Generate predicted torque

In [175]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [200]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [196]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    0.986192165056305  ⎤
⎢                           ⎥
⎢L1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢L1xz  -7.76343556708024e-27⎥
⎢                           ⎥
⎢L1yy    0.986192165056345  ⎥
⎢                           ⎥
⎢L1yz  -2.12133368993413e-16⎥
⎢                           ⎥
⎢L1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢l1x   2.40588966703881e-27 ⎥
⎢                           ⎥
⎢l1y   4.76358117877349e-17 ⎥
⎢                           ⎥
⎢l1z     0.144175828054554  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢L2xx   0.0194654973664062  ⎥
⎢                           ⎥
⎢L2xy   0.00876972571924054 ⎥
⎢                           ⎥
⎢L2xz   0.00593865075240333 ⎥
⎢                           ⎥
⎢L2yy   0.0330003696264969  ⎥
⎢         

In [178]:
from identification import barycentric2standard_params

In [179]:
sdp_opt_std.x_result

array([ 9.86192165e-01, -1.38516767e-14, -7.76343557e-27,  9.86192165e-01,
       -2.12133369e-16,  5.58233934e-05,  2.40588967e-27,  4.76358118e-17,
        1.44175828e-01,  1.79307212e+00,  4.19135562e-02,  4.97210098e-02,
       -6.98865409e-02,  1.94654974e-02,  8.76972572e-03,  5.93865075e-03,
        3.30003696e-02,  9.67284032e-04,  5.04838084e-02, -4.02232774e-01,
        6.80708808e-02,  6.43105023e-04,  5.00027870e+00,  1.09124988e-01,
        1.99956998e-01, -2.00003749e-01,  6.19401340e-04,  8.10870769e-04,
        6.31621227e-05,  1.18055051e-03, -3.76610295e-05,  1.71030714e-03,
        4.07898714e-02, -2.79045453e-02, -2.19072695e-03,  1.41404147e+00,
        1.00110498e-03,  9.95052980e-04,  2.00008574e-01,  2.02184077e-02,
        3.45713217e-03,  1.34006236e-03,  1.24674614e-02,  2.77597880e-03,
        1.57090952e-03, -8.13941900e-04,  1.04485762e-02, -1.29595950e-02,
        1.33525052e-01,  4.08774176e-02,  1.22874520e-02, -1.04736499e-04,
        7.97410096e-04, -

In [180]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [181]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    0.974599396882406  ⎤
⎢                           ⎥
⎢I1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢I1xz  -7.56998482337162e-27⎥
⎢                           ⎥
⎢I1yy    0.974599396882446  ⎥
⎢                           ⎥
⎢I1yz  -2.08303108884751e-16⎥
⎢                           ⎥
⎢I1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢r1x   1.34176960395484e-27 ⎥
⎢                           ⎥
⎢r1y   2.65665899779873e-17 ⎥
⎢                           ⎥
⎢r1z    0.0804071551405432  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢I2xx   0.0185387373447571  ⎥
⎢                           ⎥
⎢I2xy   0.00329396309376292 ⎥
⎢                           ⎥
⎢I2xz   0.00588691805246235 ⎥
⎢                           ⎥
⎢I2yy  0.000643849538560864 ⎥
⎢         

In [182]:
sdp_constraints